# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(["CustomerID", "ProductName"]).Quantity.sum().reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df2 = pd.pivot_table(df, values = "Quantity", index = "ProductName", columns = "CustomerID", fill_value = 0)
df2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pdist(df2.T, 'euclidean')
print(squareform(distances))

[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), 
                         index=df2.columns, columns=df2.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [8]:
df33 = distances[33].sort_values(ascending = False)[1:6].reset_index()
df33 = [i for i in df33.CustomerID]
df33 

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
df3 = df[(df.CustomerID == 264) | (df.CustomerID == 3535) | (df.CustomerID == 3317) | (df.CustomerID == 2503) | (df.CustomerID == 3305)].reset_index(drop=True)
df3

,CustomerID,ProductName,Quantity
0,264,Apricots - Halves,1
1,264,Apricots Fresh,1
2,264,Bacardi Breezer - Tropical,1
3,264,Bagel - Plain,1
4,264,Banana - Leaves,1
...,...,...,...
265,3535,Tray - 16in Rnd Blk,1
266,3535,Wanton Wrap,1
267,3535,Wine - Blue Nun Qualitatswein,1
268,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
df4 = df3.groupby("ProductName").Quantity.sum().sort_values(ascending = False).reset_index()
df4

,ProductName,Quantity
0,Butter - Unsalted,3
1,Soup - Campbells Bean Medley,3
2,Towels - Paper / Kraft,3
3,Wine - Ej Gallo Sierra Valley,3
4,"Pepper - Black, Whole",3
...,...,...
213,"Pepper - White, Ground",1
214,"Pepper - Paprika, Hungarian",1
215,Pate - Cognac,1
216,Pastry - Choclate Baked,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
dfcust = df[df.CustomerID == 33]
dfrec = df4.merge(dfcust, how="left", left_on='ProductName', right_on='ProductName')
dfrec = dfrec[dfrec.Quantity_y.isnull()].sort_values(by="Quantity_x", ascending = False)[0:5].reset_index(drop=True)
display(dfrec[["ProductName", "Quantity_x"]])

,ProductName,Quantity_x
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Wine - Blue Nun Qualitatswein,3
3,Soup - Campbells Bean Medley,3
4,Bacardi Breezer - Tropical,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
rec_all = {}
unique = df.CustomerID.unique()
for i in unique:
    dfi = distances[i].sort_values(ascending = False)[1:6].reset_index()
    dfi = [j for j in dfi.CustomerID]
    df3 = df[(df.CustomerID == dfi[0]) | (df.CustomerID == dfi[1]) | (df.CustomerID == dfi[2]) | (df.CustomerID == dfi[3]) | (df.CustomerID == dfi[4])].reset_index(drop=True)
    df4 = df3.groupby("ProductName").Quantity.sum().sort_values(ascending = False).reset_index()
    dfcust = df[df.CustomerID == i]
    dfrec = df4.merge(dfcust, how="left", left_on='ProductName', right_on='ProductName')
    dfrec = dfrec[dfrec.Quantity_y.isnull()].sort_values(by="Quantity_x", ascending = False)[0:5].reset_index(drop=True)
    rec_all[i] = [i for i in dfrec.ProductName]

rec_all

{33: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Wine - Blue Nun Qualitatswein',
  'Soup - Campbells Bean Medley',
  'Bacardi Breezer - Tropical'],
 200: ['Soup - Campbells Bean Medley',
  'Bay Leaf',
  'Pork - Kidney',
  'Wanton Wrap',
  'Muffin - Carrot Individual Wrap'],
 264: ['Fish - Scallops, Cold Smoked',
  'Wine - Two Oceans Cabernet',
  'Bread - Italian Roll With Herbs',
  'Fondant - Icing',
  'Veal - Inside, Choice'],
 356: ['Veal - Inside, Choice',
  'Tart Shells - Sweet, 4',
  'Pomello',
  'Beef - Inside Round',
  'Sardines'],
 412: ['Sauce - Gravy, Au Jus, Mix',
  'Olive - Spread Tapenade',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein',
  'Sprouts - Baby Pea Tendrils'],
 464: ['Butter - Unsalted',
  'Sausage - Breakfast',
  'Sauce - Gravy, Au Jus, Mix',
  'Veal - Inside, Choice',
  'Pepper - Black, Whole'],
 477: ['Cheese - Cambozola',
  'Olive - Spread Tapenade',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein',
  'Sprouts - Baby

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
store=pd.DataFrame.from_dict(rec_all)
store.head()

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Soup - Campbells Bean Medley,"Fish - Scallops, Cold Smoked","Veal - Inside, Choice","Sauce - Gravy, Au Jus, Mix",Butter - Unsalted,Cheese - Cambozola,Wine - Ej Gallo Sierra Valley,"Pepper - Black, Whole","Veal - Inside, Choice",...,Cheese - Wine,Muffin Batt - Choc Chk,"Pork - Loin, Bone - In",Grenadine,"Placemat - Scallop, White","Soup - Campbells, Lentil",Skirt - 29 Foot,"Lamb - Whole, Fresh",Pernod,Beans - Wax
1,Wine - Ej Gallo Sierra Valley,Bay Leaf,Wine - Two Oceans Cabernet,"Tart Shells - Sweet, 4",Olive - Spread Tapenade,Sausage - Breakfast,Olive - Spread Tapenade,Cod - Black Whole Fillet,"Peas - Pigeon, Dry",Wine - Blue Nun Qualitatswein,...,Muffin - Carrot Individual Wrap,Papayas,Potatoes - Idaho 100 Count,Fenngreek Seed,Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Beans - Kidney White,Bread - Raisin Walnut Oval,Crackers - Trio,Sherry - Dry
2,Wine - Blue Nun Qualitatswein,Pork - Kidney,Bread - Italian Roll With Herbs,Pomello,"Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole",Soupfoamcont12oz 112con,Lamb - Ground,"Pepper - Black, Whole",...,Meldea Green Tea Liquor,Meldea Green Tea Liquor,Pants Custom Dry Clean,Halibut - Fletches,Tofu - Firm,Wiberg Super Cure,Milk - 1%,Chips Potato Salt Vinegar 43g,Tea - Jasmin Green,"Soup - Campbells, Cream Of"
3,Soup - Campbells Bean Medley,Wanton Wrap,Fondant - Icing,Beef - Inside Round,Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice",Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Butter - Unsalted,Wine - Ej Gallo Sierra Valley,...,"Lentils - Red, Dry",Wine - Chablis 2003 Champs,Cake - Box Window 10x10x2.5,Ice Cream Bar - Oreo Cone,"Cheese - Boursin, Garlic / Herbs",Pants Custom Dry Clean,Cheese - Taleggio D.o.p.,Tea - English Breakfast,Pastry - Choclate Baked,Pears - Bosc
4,Bacardi Breezer - Tropical,Muffin - Carrot Individual Wrap,"Veal - Inside, Choice",Sardines,Sprouts - Baby Pea Tendrils,"Pepper - Black, Whole",Sprouts - Baby Pea Tendrils,Wine - Crozes Hermitage E.,Bar - Granola Trail Mix Fruit Nut,Sardines,...,Coffee - Dark Roast,Mussels - Frozen,"Veal - Brisket, Provimi,bnls","Wine - Red, Colio Cabernet",Sobe - Tropical Energy,Shrimp - 31/40,Yoghurt Tubes,Curry Paste - Madras,"Peas - Pigeon, Dry",Ecolab - Mikroklene 4/4 L


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
dist = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'jaccard'))), index=df2.columns, columns=df2.columns)

In [15]:
recom_all = {}
unique = df.CustomerID.unique()
for i in unique:
    dfi = dist[i].sort_values(ascending = False)[1:6].reset_index()
    dfi = [j for j in dfi.CustomerID]
    df3 = df[(df.CustomerID == dfi[0]) | (df.CustomerID == dfi[1]) | (df.CustomerID == dfi[2]) | (df.CustomerID == dfi[3]) | (df.CustomerID == dfi[4])].reset_index(drop=True)
    df4 = df3.groupby("ProductName").Quantity.sum().sort_values(ascending = False).reset_index()
    dfcust = df[df.CustomerID == i]
    dfrec = df4.merge(dfcust, how="left", left_on='ProductName', right_on='ProductName')
    dfrec = dfrec[dfrec.Quantity_y.isnull()].sort_values(by="Quantity_x", ascending = False)[0:5].reset_index(drop=True)
    recom_all[i] = [i for i in dfrec.ProductName]

recom_all

{33: ['Loquat',
  'Cheese - Brie, Triple Creme',
  'Blueberries',
  'Wine - Wyndham Estate Bin 777',
  'Scampi Tail'],
 200: ['Muffin - Carrot Individual Wrap',
  'Beans - Kidney, Canned',
  'Pomello',
  'Chips Potato Salt Vinegar 43g',
  'Wine - Blue Nun Qualitatswein'],
 264: ['Sauce - Demi Glace',
  'Wine - Two Oceans Cabernet',
  'Flavouring - Orange',
  'Lamb - Pieces, Diced',
  'Beer - Blue'],
 356: ['Veal - Inside, Choice',
  'Bread - Italian Roll With Herbs',
  'Tart Shells - Sweet, 4',
  'Beef - Inside Round',
  'Pepper - Black, Whole'],
 412: ['Cumin - Whole',
  'Smirnoff Green Apple Twist',
  'Pepper - Black, Whole',
  'Tart Shells - Sweet, 4',
  'Guinea Fowl'],
 464: ['Cheese - Mozzarella',
  'Sausage - Breakfast',
  'Pomello',
  'Chestnuts - Whole,canned',
  'Wine - Toasted Head'],
 477: ['Wine - Gato Negro Cabernet',
  'Cheese - Parmesan Grated',
  'Sugar - Fine',
  'Ice Cream Bar - Hageen Daz To',
  'Salmon Steak - Cohoe 8 Oz'],
 639: ['Bacardi Breezer - Tropical',
  'Mu

In [16]:
recom=pd.DataFrame.from_dict(recom_all)
recom.head()

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Loquat,Muffin - Carrot Individual Wrap,Sauce - Demi Glace,"Veal - Inside, Choice",Cumin - Whole,Cheese - Mozzarella,Wine - Gato Negro Cabernet,Bacardi Breezer - Tropical,Tea - Jasmin Green,Cod - Black Whole Fillet,...,Sugar - Fine,Rum - Mount Gay Eclipes,Coffee - Irish Cream,"Cheese - Brie,danish",Fenngreek Seed,Tray - 16in Rnd Blk,Spinach - Baby,Chocolate - Compound Coating,Coffee - Irish Cream,Muffin - Carrot Individual Wrap
1,"Cheese - Brie, Triple Creme","Beans - Kidney, Canned",Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,Smirnoff Green Apple Twist,Sausage - Breakfast,Cheese - Parmesan Grated,"Mushrooms - Black, Dried",Bread - Italian Roll With Herbs,Beans - Wax,...,Langers - Ruby Red Grapfruit,Juice - Orange,Garlic,Cake - Box Window 10x10x2.5,Vinegar - Sherry,Pernod,Beef - Ground Medium,Apricots - Halves,Soup V8 Roasted Red Pepper,Browning Caramel Glace
2,Blueberries,Pomello,Flavouring - Orange,"Tart Shells - Sweet, 4","Pepper - Black, Whole",Pomello,Sugar - Fine,Soup Knorr Chili With Beans,"Chestnuts - Whole,canned",Smirnoff Green Apple Twist,...,Wine - Ej Gallo Sierra Valley,Spice - Peppercorn Melange,Turkey - Oven Roast Breast,Wine - Magnotta - Belpaese,Bread - Italian Corn Meal Poly,Pastry - Choclate Baked,Cheese - Parmesan Cubes,Mustard Prepared,Otomegusa Dashi Konbu,Beef - Top Sirloin
3,Wine - Wyndham Estate Bin 777,Chips Potato Salt Vinegar 43g,"Lamb - Pieces, Diced",Beef - Inside Round,"Tart Shells - Sweet, 4","Chestnuts - Whole,canned",Ice Cream Bar - Hageen Daz To,Jagermeister,Skirt - 29 Foot,Scampi Tail,...,Veal - Slab Bacon,Beef - Ground Medium,Wine - Chardonnay South,Wine - Valpolicella Masi,Pastry - Choclate Baked,Ecolab - Lime - A - Way 4/4 L,Tea - Jasmin Green,Tea - Jasmin Green,"Nut - Pistachio, Shelled",Oil - Shortening - All - Purpose
4,Scampi Tail,Wine - Blue Nun Qualitatswein,Beer - Blue,"Pepper - Black, Whole",Guinea Fowl,Wine - Toasted Head,Salmon Steak - Cohoe 8 Oz,Flour - Teff,Bouq All Italian - Primerba,Kellogs Special K Cereal,...,Veal - Eye Of Round,Apricots - Halves,"Wine - White, Colubia Cresh",Island Oasis - Mango Daiquiri,"Chocolate - Semi Sweet, Calets","Beans - Kidney, Canned",Bread - French Baquette,Bread Crumbs - Panko,Muffin - Carrot Individual Wrap,Isomalt
